In [3]:
import ccxt
import numpy as np
import pandas as pd
import time
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Dropout, Bidirectional, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Конфигурация модели
TIMEFRAMES = ['1m', '5m', '15m', '1h', '4h', '1d', '1w']
LOOKBACK_PERIODS = {
    '1m': 60,
    '5m': 12,
    '15m': 4,
    '1h': 24,
    '4h': 168,
    '1d': 30,
    '1w': 12
}
FEATURES = ['open', 'high', 'low', 'close', 'volume']
NUM_FEATURES = len(FEATURES)

def fetch_multitimeframe_data(symbol='BTC/USDT', exchange_name='binance'):
    exchange = getattr(ccxt, exchange_name)({
        'enableRateLimit': True,
        'options': {'defaultType': 'spot'}
    })

    exchange.load_markets()
    data = {}

    for tf in TIMEFRAMES:
        try:
            print(f"Загрузка {tf} данных...")
            ohlcv = exchange.fetch_ohlcv(symbol, tf, limit=1000)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            data[tf] = df.dropna()
            time.sleep(exchange.rateLimit / 1000)
        except Exception as e:
            print(f"Ошибка при загрузке {tf}: {str(e)}")
            continue

    return data

def align_datasets(raw_data):
    min_samples = min([len(raw_data[tf]) - LOOKBACK_PERIODS[tf] for tf in TIMEFRAMES if tf in raw_data]) - 1

    aligned_data = {}
    scalers = {}

    for tf in TIMEFRAMES:
        if tf not in raw_data or len(raw_data[tf]) < LOOKBACK_PERIODS[tf] + min_samples:
            continue

        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(raw_data[tf][FEATURES])
        scalers[tf] = scaler

        X = []
        start_idx = len(scaled_data) - min_samples - LOOKBACK_PERIODS[tf]
        for i in range(start_idx, len(scaled_data) - LOOKBACK_PERIODS[tf]):
            X.append(scaled_data[i:i+LOOKBACK_PERIODS[tf]])

        aligned_data[tf] = np.array(X)

    return aligned_data, scalers, min_samples

def prepare_training_data(datasets, min_samples):
    X_train = {tf: data[:min_samples-1] for tf, data in datasets.items()}

    y_price = datasets['1m'][1:min_samples, -1, 3]
    y_signal = (datasets['1m'][1:min_samples, -1, 3] > datasets['1m'][:min_samples-1, -1, 3]).astype(int)

    return X_train, {'price_prediction': y_price, 'signal': y_signal}

def create_model(input_shapes):
    inputs = []
    branches = []

    for tf, shape in input_shapes.items():
        inp = Input(shape=shape, name=f'input_{tf}')
        x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)))(inp)
        x = Dropout(0.4)(x)
        x = LSTM(64, kernel_regularizer=l2(0.01))(x)
        branches.append(x)
        inputs.append(inp)

    merged = concatenate(branches)

    attention = Dense(len(branches)*64, activation='softmax')(merged)
    attended = multiply([merged, attention])

    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(attended)
    x = Dropout(0.5)(x)

    reg_output = Dense(1, name='price_prediction')(x)
    class_output = Dense(1, activation='sigmoid', name='signal')(x)

    model = Model(inputs=inputs, outputs=[reg_output, class_output])
    model.compile(
        optimizer=Adam(0.0005),
        loss={'price_prediction': 'mse', 'signal': 'binary_crossentropy'},
        metrics={'price_prediction': ['mae'], 'signal': ['accuracy']}
    )
    return model

def generate_signal(current_price, predicted_price, threshold=0.001):
    price_change = (predicted_price - current_price) / current_price
    if price_change > threshold:
        return 'buy'
    elif price_change < -threshold:
        return 'sell'
    else:
        return 'hold'

if __name__ == "__main__":
    raw_data = fetch_multitimeframe_data()
    datasets, scalers, min_samples = align_datasets(raw_data)

    if not datasets:
        raise ValueError("Недостаточно данных для обучения")

    print(f"Используется {min_samples-1} образцов для обучения")

    X_train, y_train = prepare_training_data(datasets, min_samples)

    input_shapes = {tf: (LOOKBACK_PERIODS[tf], NUM_FEATURES) for tf in datasets}
    model = create_model(input_shapes)

    X_train_named = {f'input_{tf}': data for tf, data in X_train.items()}

    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

    history = model.fit(
        X_train_named,
        y_train,
        epochs=50,
        batch_size=64,
        validation_split=0.2,
        verbose=1,
        callbacks=[lr_scheduler]
    )

    print("Прогнозирование...")

    current_data = {}
    for tf in datasets:
        last_seq = datasets[tf][-1]
        current_data[f'input_{tf}'] = np.expand_dims(last_seq, axis=0)

    price_pred_scaled, signal_prob = model.predict(current_data)

    # Инвертирование стандартизации для прогнозируемой цены
    scaler_1m = scalers['1m']
    dummy = np.zeros((1, NUM_FEATURES))
    dummy[0, 3] = price_pred_scaled[0][0]
    price_pred_unscaled = scaler_1m.inverse_transform(dummy)[0, 3]

    signal = generate_signal(
        current_price=raw_data['1m'].iloc[-1]['close'],
        predicted_price=price_pred_unscaled,
        threshold=0.002
    )

    print("\nРезультаты:")
    print(f"Текущая цена: {raw_data['1m'].iloc[-1]['close']}")
    print(f"Прогнозируемая цена: {price_pred_unscaled}")
    print(f"Вероятность роста: {signal_prob[0][0]:.4f}")
    print(f"Рекомендуемое действие: {signal.upper()}")

    print("\nМетрики валидации (последняя эпоха):")
    print(f"Val MAE: {history.history['val_price_prediction_mae'][-1]:.4f}")
    print(f"Val Accuracy: {history.history['val_signal_accuracy'][-1]:.4f}")
    print(f"Val Price Loss: {history.history['val_price_prediction_loss'][-1]:.4f}")
    print(f"Val Signal Loss: {history.history['val_signal_loss'][-1]:.4f}")

Загрузка 1m данных...
Загрузка 5m данных...
Загрузка 15m данных...
Загрузка 1h данных...
Загрузка 4h данных...
Загрузка 1d данных...
Загрузка 1w данных...
Используется 393 образцов для обучения
Epoch 1/50
5/5 [==============================] - 52s 4s/step - loss: 24.4998 - price_prediction_loss: 1.8455 - signal_loss: 0.6931 - price_prediction_mae: 1.2294 - signal_accuracy: 0.5191 - val_loss: 23.6686 - val_price_prediction_loss: 1.9568 - val_signal_loss: 0.6933 - val_price_prediction_mae: 1.3858 - val_signal_accuracy: 0.4430 - lr: 5.0000e-04
Epoch 2/50
5/5 [==============================] - 12s 2s/step - loss: 22.9278 - price_prediction_loss: 1.8045 - signal_loss: 0.6929 - price_prediction_mae: 1.2192 - signal_accuracy: 0.5510 - val_loss: 22.1105 - val_price_prediction_loss: 1.8808 - val_signal_loss: 0.6934 - val_price_prediction_mae: 1.3581 - val_signal_accuracy: 0.4430 - lr: 5.0000e-04
Epoch 3/50
5/5 [==============================] - 11s 2s/step - loss: 21.4330 - price_prediction_los